<a href="https://colab.research.google.com/github/Ani287/TextGenLSTM/blob/master/MachineLearningMastery_Tutorial_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.1'

In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [1]:
!pip install tensorflow-gpu

     |████████████████████████████████| 516.2MB 32kB/s 


In [2]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [3]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [7]:
import pandas as pd 
from google.colab import files
file = files.upload() 

Saving grimms_fairytales.txt to grimms_fairytales.txt


In [10]:
f = open("grimms_fairytales.txt", encoding="utf8")
document = f.read().lower()
f.close()
print('Corpus length:', len(document)) #545009
print(document[:1000])

Corpus length: 544009
,title,text			
"0,the golden bird,""a certain king had a beautiful garden, and in the garden stood a tree"			
which bore golden apples. these apples were always counted, and about			
the time when they began to grow ripe it was found that every night one			
of them was gone. the king became very angry at this, and ordered the			
gardener to keep watch all night under the tree. the gardener set his			
eldest son to watch	 but about twelve o’clock he fell asleep, and in		
the morning another of the apples was missing. then the second son was			
ordered to watch	 and at midnight he too fell asleep, and in the morning		
another apple was gone. then the third son offered to keep watch	 but		
the gardener at first would not let him, for fear some harm should come			
to him: however, at last he consented, and the young man laid himself			
under the tree to watch. as the clock struck twelve he heard a rustling			
noise in the air, and a bird came flying that was of pure g

In [11]:
document_new = document.split()
fairytales = " ".join(document_new)

x = fairytales.find("*****")
print(x)

text = fairytales[12:515323] 
print(text[:1000])

515323
"0,the golden bird,""a certain king had a beautiful garden, and in the garden stood a tree" which bore golden apples. these apples were always counted, and about the time when they began to grow ripe it was found that every night one of them was gone. the king became very angry at this, and ordered the gardener to keep watch all night under the tree. the gardener set his eldest son to watch but about twelve o’clock he fell asleep, and in the morning another of the apples was missing. then the second son was ordered to watch and at midnight he too fell asleep, and in the morning another apple was gone. then the third son offered to keep watch but the gardener at first would not let him, for fear some harm should come to him: however, at last he consented, and the young man laid himself under the tree to watch. as the clock struck twelve he heard a rustling noise in the air, and a bird came flying that was of pure gold and as it was snapping at one of the apples with its beak, the

In [12]:
#text = raw_text in tut

# create mapping of unique chars to integers
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

char_to_int

{' ': 0,
 '!': 1,
 '"': 2,
 '(': 3,
 ')': 4,
 '*': 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '4': 13,
 '5': 14,
 '6': 15,
 '7': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 '?': 20,
 '[': 21,
 ']': 22,
 '_': 23,
 'a': 24,
 'b': 25,
 'c': 26,
 'd': 27,
 'e': 28,
 'f': 29,
 'g': 30,
 'h': 31,
 'i': 32,
 'j': 33,
 'k': 34,
 'l': 35,
 'm': 36,
 'n': 37,
 'o': 38,
 'p': 39,
 'q': 40,
 'r': 41,
 's': 42,
 't': 43,
 'u': 44,
 'v': 45,
 'w': 46,
 'x': 47,
 'y': 48,
 'z': 49,
 '‘': 50,
 '’': 51,
 '“': 52,
 '”': 53}

In [13]:
# Now that the book has been loaded and the mapping prepared, we can summarize the dataset

n_chars = len(text)
n_vocab = len(chars)
print("Total Characters: ", n_chars) 
print("Total Vocab: ", n_vocab) #54 distinct characters

Total Characters:  515311
Total Vocab:  54


In [14]:
# need to define the training data for the network
# a lot of flexibility in how you choose to break up the text and expose it to the network during training
# here: the text is split up into subsequences with a fixed length of 100 characters, an arbitrary length

# Each training pattern of the network is comprised of 100 time steps of one character (X) followed by one 
# character output (y)
# When creating these sequences, we slide this window along the whole book one character at a time, allowing 
# each character a chance to be learned from the 100 characters that preceded it (except the first 100 characters of course)

# As we split up the book into these sequences, we convert the characters to integers using our lookup table we prepared earlier

# prepare the dataset of input to output pairs encoded as integers

seq_length = 100 
dataX = []
dataY = []

for i in range (0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

# Now that we have prepared our training data we need to transform it so that it is suitable for use with Keras

Total Patterns:  515211


In [ ]:
# to do

# 1. transform the list of input sequences into the form [samples, time steps, features] expected by an LSTM network
# 2. rescale the integers to the range 0-to-1 to make the patterns easier to learn by the LSTM network --> sigmoid function by default
# 3.  convert the output patterns (single characters converted to integers) into a one hot encoding

# why? so that we can configure the network to predict the probability of each of the 54 different characters in the 
# vocabulary (an easier representation) rather than trying to force it to predict precisely the next character

# Each y value is converted into a sparse vector with a length of 54, full of zeros except with a 1 in the column 
# for the letter (integer) that the pattern represents.

In [15]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

# now can define our LSTM model

In [16]:
# define a single hidden LSTM layer with 256 memory units
# The network uses dropout with a probability of 20
# The output layer is a Dense layer using the softmax activation function to output a probability prediction 
# for each of the 54 characters between 0 and 1

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [19]:
# There is no test dataset. We are modeling the entire training dataset to learn the probability of each character in a sequence

# NOT interested in the most accurate (classification accuracy) model of the training dataset
# This would be a model that predicts each character in the training dataset PERFECTLY
# instead: interested in a generalization of the dataset that minimizes the chosen loss function
# seeking a balance between generalization and overfitting but short of memorization

# Because of the slowness and because of our optimization requirements --> use model checkpointing 
# to record all of the network weights to file each time an improvement in loss is observed at the end of the epoch


# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# now fit our model to the data. Here we use a modest number of 20 epochs and a large batch size of 128 patterns

In [20]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
515211/515211 [==============================] - 956s 2ms/step - loss: 2.6612

Epoch 00001: loss improved from inf to 2.66124, saving model to weights-improvement-01-2.6612.hdf5
Epoch 2/20
515211/515211 [==============================] - 955s 2ms/step - loss: 2.4493

Epoch 00002: loss improved from 2.66124 to 2.44929, saving model to weights-improvement-02-2.4493.hdf5
Epoch 3/20
515211/515211 [==============================] - 979s 2ms/step - loss: 2.3187

Epoch 00003: loss improved from 2.44929 to 2.31874, saving model to weights-improvement-03-2.3187.hdf5
Epoch 4/20
515211/515211 [==============================] - 985s 2ms/step - loss: 2.2161

Epoch 00004: loss improved from 2.31874 to 2.21610, saving model to weights-improvement-04-2.2161.hdf5
Epoch 5/20
515211/515211 [==============================] - 989s 2ms/step - loss: 2.1359

Epoch 00005: loss improved from 2.21610 to 2.13585, saving model to weights-improvement-05-2.1359.hdf5
Epoch 6/20
515211/515211 [=============

In [ ]:
# Generating text using the trained LSTM network is relatively straightforward

# load the network weights
filename = "weights-improvement-19-1.9435.hdf5" # look where loss is the smallest!
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# preparing the mapping of unique characters to integers, we must also create a reverse mapping that we can 
# use to convert the integers back to characters so that we can understand the predictions

int_to_char = dict((i, c) for i, c in enumerate(chars))


In [ ]:

# first start off with a seed sequence as input, generate the next character then update the seed 
# sequence to add the generated character on the end and trim off the first character
# This process is repeated for as long as we want to predict new characters (e.g. a sequence of 1,000 characters in length)


# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]

print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

In [ ]:
# larger model xxx

### 10 Extension Ideas to Improve the Model
Below are 10 ideas that may further improve the model that you could experiment with are:

* Predict fewer than 1,000 characters as output for a given seed.
* Remove all punctuation from the source text, and therefore from the models’ vocabulary.
* Try a one hot encoded for the input sequences.
* Train the model on padded sentences rather than random sequences of characters.
* Increase the number of training epochs to 100 or many hundreds.
* Add dropout to the visible input layer and consider tuning the dropout percentage.
* Tune the batch size, try a batch size of 1 as a (very slow) baseline and larger sizes from there.
* Add more memory units to the layers and/or more layers.
* Experiment with scale factors (temperature) when interpreting the prediction probabilities.
* Change the LSTM layers to be “stateful” to maintain state across batches.